<a href="https://colab.research.google.com/github/jumanewton/pydata/blob/main/Sequence_Models_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.regularizers import L1, L2, L1L2
from tensorflow.keras.optimizers import Adam, RMSprop

imdb_train = tfds.load(name="imdb_reviews", split="train")
imdb_train_df = tfds.as_dataframe(imdb_train)
imdb_train_df['text'] = imdb_train_df['text'].str.decode('utf-8')

imdb_test = tfds.load(name="imdb_reviews", split="test")
imdb_test_df = tfds.as_dataframe(imdb_test)
imdb_test_df['text'] = imdb_test_df['text'].str.decode('utf-8')

X_train = imdb_train_df['text']
y_train = imdb_train_df['label']
X_test = imdb_test_df['text']
y_test = imdb_test_df['label']



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.1G1Y2F_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.1G1Y2F_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.1G1Y2F_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [2]:
max_tokens = 20000
output_sequence_length = 256
output_dim = 128

vectorizer_layer = TextVectorization(max_tokens=max_tokens,
                                     output_mode='int',
                                     standardize='lower_and_strip_punctuation',
                                     ngrams=1,
                                     output_sequence_length=output_sequence_length)
vectorizer_layer.adapt(X_train)

embedding_layer = Embedding(input_dim=max_tokens,
                            output_dim=output_dim,
                            input_length=output_sequence_length)

In [3]:
simple_lstm = Sequential()
simple_lstm.add(Input(shape=(1,), dtype=tf.string))
simple_lstm.add(vectorizer_layer)
simple_lstm.add(embedding_layer)
simple_lstm.add(LSTM(128))
simple_lstm.add(Dense(64, activation='relu'))
simple_lstm.add(Dense(1, activation='sigmoid'))

opt=Adam(learning_rate=0.001)

simple_lstm.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
simple_lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 256)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 256, 128)          2560000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2699905 (10.30 MB)
Trainable params: 2699905 (10.30 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [4]:
simple_lstm.fit(X_train, y_train, epochs=5)



Epoch 1/5
782/782 [==============================] - 54s 63ms/step - loss: 0.6918 - accuracy: 0.5216
Epoch 2/5
782/782 [==============================] - 20s 26ms/step - loss: 0.6779 - accuracy: 0.5560
Epoch 3/5
782/782 [==============================] - 16s 20ms/step - loss: 0.4608 - accuracy: 0.7680
Epoch 4/5
782/782 [==============================] - 15s 19ms/step - loss: 0.2376 - accuracy: 0.9115
Epoch 5/5
782/782 [==============================] - 14s 18ms/step - loss: 0.1436 - accuracy: 0.9514


In [5]:
test_loss, test_acc = simple_lstm.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

782/782 [==============================] - 7s 9ms/step - loss: 0.4373 - accuracy: 0.8481
Test Loss: 0.4372574985027313
Test Accuracy: 0.8480799794197083


In [6]:
sl_bi_lstm = Sequential()
sl_bi_lstm.add(Input(shape=(1,), dtype=tf.string))
sl_bi_lstm.add(vectorizer_layer)
sl_bi_lstm.add(embedding_layer)
sl_bi_lstm.add(Bidirectional(LSTM(128)))
sl_bi_lstm.add(Dense(64, activation='relu', kernel_regularizer=L1L2(l1=0.005,l2=0.005)))
sl_bi_lstm.add(Dense(32, activation='relu', kernel_regularizer=L2(l2=0.005)))
sl_bi_lstm.add(Dense(16, activation='relu', kernel_regularizer=L2(l2=0.01)))
sl_bi_lstm.add(Dense(1, activation='sigmoid'))

opt=Adam(learning_rate=0.001)

sl_bi_lstm.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
sl_bi_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 256)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 256, 128)          2560000   
                                                                 
 bidirectional (Bidirection  (None, 256)               263168    
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 16)               

In [7]:
sl_bi_lstm.fit(X_train, y_train, epochs=3)

Epoch 1/3
782/782 [==============================] - 59s 69ms/step - loss: 1.0432 - accuracy: 0.7976
Epoch 2/3
782/782 [==============================] - 26s 34ms/step - loss: 0.5398 - accuracy: 0.8136
Epoch 3/3
782/782 [==============================] - 23s 29ms/step - loss: 0.4064 - accuracy: 0.8686


In [8]:
test_loss, test_acc = sl_bi_lstm.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

782/782 [==============================] - 11s 12ms/step - loss: 0.4335 - accuracy: 0.8544
Test Loss: 0.4335095286369324
Test Accuracy: 0.8543599843978882


In [9]:
ml_bi_lstm = Sequential()
ml_bi_lstm.add(Input(shape=(1,), dtype=tf.string))
ml_bi_lstm.add(vectorizer_layer)
ml_bi_lstm.add(embedding_layer)
ml_bi_lstm.add(Bidirectional(LSTM(4, return_sequences=True)))
ml_bi_lstm.add(Bidirectional(LSTM(4, return_sequences=True)))
ml_bi_lstm.add(Bidirectional(LSTM(4)))
ml_bi_lstm.add(Dense(8, activation='elu', kernel_regularizer=L1L2(l1=0.1,l2=0.1)))
ml_bi_lstm.add(Dense(8, activation='elu', kernel_regularizer=L2(l2=0.1)))
ml_bi_lstm.add(Dense(8, activation='elu', kernel_regularizer=L2(l2=0.05)))
ml_bi_lstm.add(Dense(1, activation='sigmoid'))

opt=RMSprop(learning_rate=0.0001, rho=0.8, momentum=0.9)

ml_bi_lstm.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
ml_bi_lstm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 256)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 256, 128)          2560000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 256, 8)            4256      
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 256, 8)            416       
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 8)                 416       
 onal)                                                

In [10]:
ml_bi_lstm.fit(X_train, y_train, epochs=3)

Epoch 1/3
782/782 [==============================] - 78s 88ms/step - loss: 1.5504 - accuracy: 0.8641
Epoch 2/3
782/782 [==============================] - 46s 59ms/step - loss: 0.5304 - accuracy: 0.9134
Epoch 3/3
782/782 [==============================] - 42s 54ms/step - loss: 0.4252 - accuracy: 0.9299


In [11]:
test_loss, test_acc = ml_bi_lstm.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

782/782 [==============================] - 21s 25ms/step - loss: 0.5934 - accuracy: 0.8338
Test Loss: 0.593397319316864
Test Accuracy: 0.833840012550354


In [14]:
ml_bi_lstm_sum = Sequential()
ml_bi_lstm_sum.add(Input(shape=(1,), dtype=tf.string))
ml_bi_lstm_sum.add(vectorizer_layer)
ml_bi_lstm_sum.add(embedding_layer)
ml_bi_lstm_sum.add(Bidirectional(LSTM(8, return_sequences=True), merge_mode='sum'))
ml_bi_lstm_sum.add(Bidirectional(LSTM(4)))
ml_bi_lstm_sum.add(Dense(16, activation='relu'))
ml_bi_lstm_sum.add(tf.keras.layers.Dropout(0.5))
ml_bi_lstm_sum.add(Dense(1, activation='sigmoid'))

opt=Adam(learning_rate=0.001)

ml_bi_lstm_sum.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
ml_bi_lstm_sum.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 256)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 256, 128)          2560000   
                                                                 
 bidirectional_6 (Bidirecti  (None, 256, 8)            8768      
 onal)                                                           
                                                                 
 bidirectional_7 (Bidirecti  (None, 8)                 416       
 onal)                                                           
                                                                 
 dense_11 (Dense)            (None, 16)                144       
                                                      

In [15]:
ml_bi_lstm_sum.fit(X_train, y_train, epochs=3)

test_loss, test_acc = ml_bi_lstm_sum.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

Epoch 1/3
782/782 [==============================] - 71s 80ms/step - loss: 0.3293 - accuracy: 0.8740
Epoch 2/3
782/782 [==============================] - 36s 46ms/step - loss: 0.2055 - accuracy: 0.9355
Epoch 3/3
782/782 [==============================] - 16s 19ms/step - loss: 0.4697 - accuracy: 0.8408
Test Loss: 0.4696926176548004
Test Accuracy: 0.8407599925994873


In [16]:
ml_bi_lstm_ave = Sequential()
ml_bi_lstm_ave.add(Input(shape=(1,), dtype=tf.string))
ml_bi_lstm_ave.add(vectorizer_layer)
ml_bi_lstm_ave.add(embedding_layer)
ml_bi_lstm_ave.add(Bidirectional(LSTM(8, return_sequences=True), merge_mode='ave'))
ml_bi_lstm_ave.add(Bidirectional(LSTM(4)))
ml_bi_lstm_ave.add(Dense(16, activation='relu'))
ml_bi_lstm_ave.add(tf.keras.layers.Dropout(0.5))
ml_bi_lstm_ave.add(Dense(1, activation='sigmoid'))

opt=Adam(learning_rate=0.001)

ml_bi_lstm_ave.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
ml_bi_lstm_ave.summary()

ml_bi_lstm_ave.fit(X_train, y_train, epochs=3)

test_loss, test_acc = ml_bi_lstm_ave.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 256)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 256, 128)          2560000   
                                                                 
 bidirectional_8 (Bidirecti  (None, 256, 8)            8768      
 onal)                                                           
                                                                 
 bidirectional_9 (Bidirecti  (None, 8)                 416       
 onal)                                                           
                                                                 
 dense_13 (Dense)            (None, 16)                144       
                                                      

In [17]:
ml_bi_lstm_mul = Sequential()
ml_bi_lstm_mul.add(Input(shape=(1,), dtype=tf.string))
ml_bi_lstm_mul.add(vectorizer_layer)
ml_bi_lstm_mul.add(embedding_layer)
ml_bi_lstm_mul.add(Bidirectional(LSTM(8, return_sequences=True), merge_mode='mul'))
ml_bi_lstm_mul.add(Bidirectional(LSTM(4)))
ml_bi_lstm_mul.add(Dense(16, activation='relu'))
ml_bi_lstm_mul.add(tf.keras.layers.Dropout(0.5))
ml_bi_lstm_mul.add(Dense(1, activation='sigmoid'))

opt=Adam(learning_rate=0.001)

ml_bi_lstm_mul.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
ml_bi_lstm_mul.summary()

ml_bi_lstm_mul.fit(X_train, y_train, epochs=3)

test_loss, test_acc = ml_bi_lstm_mul.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 256)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 256, 128)          2560000   
                                                                 
 bidirectional_10 (Bidirect  (None, 256, 8)            8768      
 ional)                                                          
                                                                 
 bidirectional_11 (Bidirect  (None, 8)                 416       
 ional)                                                          
                                                                 
 dense_15 (Dense)            (None, 16)                144       
                                                      